In [2]:
import requests
import defs
import pandas as pd

In [31]:
session = requests.Session()

下面這段程式碼是為了解決最後在找所有可交易商品時出現的 ```ConnectionResetError 10054```   
詳見 [這篇文章](https://stackoverflow.com/questions/27333671/how-to-solve-the-10054-error)

In [32]:
session.headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.1.2222.33 Safari/537.36",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
}



In [4]:
ins_df = pd.read_pickle("instruments.pkl")

In [5]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [6]:
ins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [7]:
ins_df.name.unique()

array(['ZAR_JPY', 'EUR_HUF', 'EUR_DKK', 'USD_MXN', 'GBP_USD', 'CAD_CHF',
       'EUR_GBP', 'GBP_CHF', 'USD_THB', 'USD_ZAR', 'HKD_JPY', 'CHF_ZAR',
       'CAD_JPY', 'EUR_HKD', 'AUD_HKD', 'EUR_NZD', 'SGD_CHF', 'AUD_SGD',
       'EUR_JPY', 'USD_CHF', 'USD_TRY', 'GBP_JPY', 'EUR_CZK', 'EUR_TRY',
       'USD_JPY', 'GBP_ZAR', 'SGD_JPY', 'USD_NOK', 'TRY_JPY', 'USD_DKK',
       'CHF_JPY', 'EUR_PLN', 'AUD_CAD', 'USD_HKD', 'GBP_NZD', 'CAD_SGD',
       'NZD_USD', 'AUD_NZD', 'CHF_HKD', 'USD_CAD', 'USD_CNH', 'USD_CZK',
       'NZD_CHF', 'NZD_SGD', 'NZD_HKD', 'AUD_CHF', 'USD_SEK', 'GBP_SGD',
       'AUD_JPY', 'EUR_ZAR', 'EUR_AUD', 'NZD_JPY', 'GBP_HKD', 'USD_SGD',
       'EUR_SEK', 'EUR_CHF', 'EUR_CAD', 'USD_HUF', 'NZD_CAD', 'EUR_SGD',
       'AUD_USD', 'EUR_USD', 'GBP_AUD', 'USD_PLN', 'CAD_HKD', 'GBP_CAD',
       'GBP_PLN', 'EUR_NOK'], dtype=object)

In [8]:
# 列出我們 想交易 且 demo account 允許我們交易的商品
for p1 in our_curr:
    for p2 in our_curr:
        pair = f'{p1}_{p2}'
        if pair in ins_df.name.unique():
            print(pair)

EUR_USD
EUR_GBP
EUR_JPY
EUR_CHF
EUR_NZD
EUR_CAD
USD_JPY
USD_CHF
USD_CAD
GBP_USD
GBP_JPY
GBP_CHF
GBP_NZD
GBP_CAD
CHF_JPY
NZD_USD
NZD_JPY
NZD_CHF
NZD_CAD
CAD_JPY
CAD_CHF


In [9]:
def fetch_candles(pair_name, count, granularity):
    url = f'{defs.OANDA_URL}instruments/{pair_name}/candles'

    # 從 test.ipynb 直接摳過來
    params = dict(
    count = count, 
    granularity = granularity,
    price = 'MBA' # M: midpoint, B: bid, A: ask
    )

    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()


In [10]:
code, res = fetch_candles("EUR_USD", 10, 'H1')

In [11]:
type(res)

dict

In [12]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 258,
   'time': '2021-09-07T21:00:00.000000000Z',
   'bid': {'o': '1.18369', 'h': '1.18445', 'l': '1.18368', 'c': '1.18435'},
   'mid': {'o': '1.18408', 'h': '1.18460', 'l': '1.18402', 'c': '1.18449'},
   'ask': {'o': '1.18447', 'h': '1.18499', 'l': '1.18434', 'c': '1.18463'}},
  {'complete': True,
   'volume': 366,
   'time': '2021-09-07T22:00:00.000000000Z',
   'bid': {'o': '1.18433', 'h': '1.18444', 'l': '1.18394', 'c': '1.18432'},
   'mid': {'o': '1.18447', 'h': '1.18452', 'l': '1.18426', 'c': '1.18440'},
   'ask': {'o': '1.18461', 'h': '1.18471', 'l': '1.18442', 'c': '1.18448'}},
  {'complete': True,
   'volume': 257,
   'time': '2021-09-07T23:00:00.000000000Z',
   'bid': {'o': '1.18433', 'h': '1.18435', 'l': '1.18409', 'c': '1.18423'},
   'mid': {'o': '1.18442', 'h': '1.18443', 'l': '1.18416', 'c': '1.18430'},
   'ask': {'o': '1.18450', 'h': '1.18451', 'l': '1.18423', 'c': '1.18437'}},
  

### 目標:
將上一個函數回傳的 response，整理成一個 DataFrame。  
拆解，我們需要:  
- ```complete```
- ```volume```
- ```time```
- 各種價格方式的 o, h, l, c
---
想法:  
用一個 list 存放各商品的資料 (因此 list 裡放的是字典)  
再把字典轉為 data frame

In [13]:

def get_candles_df(json_response): # json_response is a dict
    our_data = []
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    for candle in json_response['candles']: # 將時間段內的 candles 逐一取出
        if candle['complete'] == False:
            continue

        newdict = {}
        newdict['time'] = candle['time']
        newdict['volume'] = candle['volume']
        for p in prices:
            for oh in ohlc:
                newdict[f'{p}_{oh}'] = candle[p][oh]
        our_data.append(newdict)
    return pd.DataFrame.from_dict(our_data)
    

In [16]:
code, res = fetch_candles("EUR_USD", 10, 'H1')
df = get_candles_df(res)

In [17]:
df.head

<bound method NDFrame.head of                              time  volume    mid_o    mid_h    mid_l    mid_c  \
0  2021-09-07T21:00:00.000000000Z     258  1.18408  1.18460  1.18402  1.18449   
1  2021-09-07T22:00:00.000000000Z     366  1.18447  1.18452  1.18426  1.18440   
2  2021-09-07T23:00:00.000000000Z     257  1.18442  1.18443  1.18416  1.18430   
3  2021-09-08T00:00:00.000000000Z     914  1.18428  1.18482  1.18405  1.18480   
4  2021-09-08T01:00:00.000000000Z     879  1.18479  1.18511  1.18456  1.18476   
5  2021-09-08T02:00:00.000000000Z     706  1.18478  1.18482  1.18446  1.18450   
6  2021-09-08T03:00:00.000000000Z     871  1.18451  1.18458  1.18406  1.18422   
7  2021-09-08T04:00:00.000000000Z     535  1.18420  1.18438  1.18413  1.18436   
8  2021-09-08T05:00:00.000000000Z    1256  1.18435  1.18436  1.18352  1.18366   

     bid_o    bid_h    bid_l    bid_c    ask_o    ask_h    ask_l    ask_c  
0  1.18369  1.18445  1.18368  1.18435  1.18447  1.18499  1.18434  1.18463  
1  1.18

## 將資料存放:

In [18]:
def save_file(candles_df, pair, granularity): # pair 與 granularity 是為了以後方便查閱資料
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

## 將上述三個 function 合起來:
- ```fetch_candles()```
- ```get_candles_df()```
- ```save_file()```

In [19]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity) # 這邊預設取 4000 筆資料，當然可以客製化
    if code != 200:
        print(pair, 'Error') # 如果連線時有錯誤，則將錯誤的商品印出來
        return 
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [22]:
create_data('EUR_USD', 'H1')

EUR_USD loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z


### 之後，就可以選擇要查詢哪一些商品的 data 了

In [25]:
from time import sleep

In [33]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f'{p1}_{p2}'
        if pair in ins_df.name.unique():
            create_data(pair, 'H1')
            

EUR_USD loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2021-01-18T15:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
USD_JPY loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
USD_CHF loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
USD_CAD loaded 3999 candles from 2021-01-18T16:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
GBP_USD loaded 3999 candles from 2021-01-18T15:00:00.000000000Z to 2021-09-08T05:00:00.000000000Z
GBP_JPY loaded 3999 